In [1]:
%matplotlib widget
import ibvpy.api as ib
import matplotlib.pylab as plt
from matplotlib import cm
from scipy.interpolate import interp2d
from scipy.signal import argrelextrema
from bmcs_shear.api import CrackBridgeAdv
from bmcs_shear.dic_crack import\
    DICGrid, DICStateFields, DICCrack, DICCrackCOR
from bmcs_shear.shear_crack.crack_path import get_T_Lab
import numpy as np
np.seterr(divide='ignore', invalid='ignore');

In [2]:
dic_grid = DICGrid(start_t=0, end_t=1, U_factor=100, dir_name='B8_TV2')

In [3]:
dsf = DICStateFields(dic_grid=dic_grid, t=0.5)
dsf.tmodel_.trait_set(E=5000, c_T=0, nu=0.18, epsilon_0=0.0005, epsilon_f=0.01);
dsf.eval()

In [4]:
dc = DICCrack(dsf=dsf)

In [5]:
dc.interact()

In [5]:
dc.dic_grid.get_F_eta_dic_idx()

13

In [6]:
dc_cor = DICCrackCOR(dic_crack=dc)

In [7]:
dc_cor.dsf

In [8]:
dc_cor.interact()